> This notebook contains exploratory code and notes written while following Andrej Karpathy’s GPT-from-scratch video.

In [123]:
with open("models/input.txt", "r", encoding="utf-8") as f:
    text = f.read()

len(text)

1115394

# **Data Preprocessing**

In [124]:
# GET ALL INDIVIDUAL CHARACTERS
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [125]:
# STRING TO INTEGER AND REVERSE MAPPING
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Hi there"))
print(decode([20, 47, 1, 58, 46, 43, 56, 43]))

[20, 47, 1, 58, 46, 43, 56, 43]
Hi there


In [126]:
# now encode entire text and make it tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
data.shape

torch.Size([1115394])

In [127]:
# train-test split the data
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

print(train_data.shape)
print(val_data.shape)

torch.Size([1003854])
torch.Size([111540])


In [128]:
# generate batches of data
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb = get_batch("train")
print(xb.shape) #[4,8] 4 separate occasions/batches of 8 continuous characters
print(yb.shape) #[4,8] predicted next character for each character in previous matrix

torch.Size([4, 8])
torch.Size([4, 8])


In [129]:
# better explanation of what is happening
print("INPUTS: \n", xb)
print("TARGETS: \n", yb)
print("-"*50+"\n")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} then target is {target}")

INPUTS: 
 tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
TARGETS: 
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
--------------------------------------------------

when input is [24] then target is 43
when input is [24, 43] then target is 58
when input is [24, 43, 58] then target is 5
when input is [24, 43, 58, 5] then target is 57
when input is [24, 43, 58, 5, 57] then target is 1
when input is [24, 43, 58, 5, 57, 1] then target is 46
when input is [24, 43, 58, 5, 57, 1, 46] then target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] then target is 39
when input is [44] then target is 53
when input is [44, 53] then target is 56
when input is [44, 53, 56] then target is 1
when input is [44, 53, 56, 1] then target is 58
when input is [44, 53

# **BigramLanguageModel**

In [130]:
#BigramLanguageModel
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLangaugeModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)

            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self,idx,max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx,idx_next), dim=1)
        return idx

m = BigramLangaugeModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [131]:
# TRAINING
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size=32
for steps in range(10000):
    xb, yb  = get_batch('train')            # batch

    logits, loss = m(xb,yb)                 # forward pass
    optimizer.zero_grad(set_to_none=True)   # zero_grad
    loss.backward()                         # backward pass
    optimizer.step()                        # optimize

print(loss.item())

2.394822359085083


In [132]:
# OVERALL EVALUATION
@torch.no_grad()
def evaluate_full_dataset(split):
    data = train_data if split == 'train' else val_data

    m.eval()
    logits, loss = m(data[:-1].unsqueeze(0), data[1:].unsqueeze(0)) # Forward Pass
    m.train()

    return loss.item()

# Calculate final losses
final_train_loss = evaluate_full_dataset('train')
final_val_loss = evaluate_full_dataset('val')

print(f"\nFinal train loss: {final_train_loss:.4f}")
print(f"Final val loss: {final_val_loss:.4f}")


Final train loss: 2.4667
Final val loss: 2.4864


In [133]:
# GENERATE
idx = torch.zeros((1,1), dtype=torch.long) # just a tensor having a single "0" to start the generation
print(decode(m.generate(idx,max_new_tokens=500)[0].tolist()))


LA c wo the;
Pancalolinghowhatharean:
QA:

Wwhass bowoond;
Fomere d shdeenotep.
CI y mbotot swefesealso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee anf,
TOFonk? me ain ckntoty dedo bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLengerithesinthengove fal amas trr
TI ar I t, mes, o sar; my w, fredeeyove
THek' merer,


## **Self-Attention**

In [134]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [136]:
# averaging previous content
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #(t,C)
        xbow[b,t] = torch.mean(xprev,0)

In [137]:
x[0] # original first batch

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [138]:
xbow[0] # averaged first batch

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

> **Now we do the Mathematical Trick rather than using for-loop**

In [148]:
# EXPLANATION OF THE MATHEMATICAL TRICK
# version 2
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a/torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print("a=")
print(a)
print("--")
print("b=")
print(b)
print("--")
print("c=")
print(c)

# so here a is (3,3)
# b is (3,2)
# c becomes (3,2) and also has averaged values

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [149]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 =  wei @ x # (T,T) @ (B,T,C) ---> (B,T,T) @ (B,T,C) ---> (B,T,C)

In [150]:
xbow[0] # xbow from previous for-loop

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [151]:
xbow2[0] # xbow from current matrix multiplication trick

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

> both are same, hence the mathematical trick works

In [179]:
# another identical method that uses Softmax
# version 3
tril = torch.tril(torch.ones(T,T))
wei  = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x

In [162]:
xbow[0] # xbow from previous for-loop

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [164]:
xbow3[0] # xbow from current softmax trick

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [183]:
# Version 4: Self Attention
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False) #(32,16)
query = nn.Linear(C, head_size, bias=False) #(32,16)
value = nn.Linear(C, head_size, bias=False)#(32,16)

k = key(x) #(4,8,32) @ (32,16) --> (4,8,16)
q = query(x)  #(4,8,32) @ (32,16) --> (4,8,16)

wei =  q @ k.transpose(-2,-1) #(4,8,16) @ (4,16,8) --> (4,8,8)

wei =  wei * head_size**-0.5


tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)


v = value(x) #(4,8,32) @ (32,16) --> (4,8,16)

out = wei @ v #(4,8,16) @ (4,8,16) --> (4,8,16)
